In [1]:
from torchvision.datasets import ImageFolder
import torch
import torchvision
import torchvision.transforms as transforms
import pandas as pd
import numpy as np

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
testset = ImageFolder("data/test_loader/", transform=transform)

FileNotFoundError: [WinError 3] System nie może odnaleźć określonej ścieżki: 'data/test_loader/'

In [ ]:
train_transform = transforms.Compose(
    [transforms.RandomHorizontalFlip(0.5),
     transforms.RandomRotation(24),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 32
trainset = ImageFolder("data/train/", transform=train_transform)
testset = ImageFolder("data/test_all/", transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=512,
                                         shuffle=False)

In [4]:
import os, shutil

In [6]:
test_mapping = pd.DataFrame()
for img, cl in testset.imgs:
    new_img_name = str(np.random.rand())[2:]
    new_row = pd.DataFrame({"img": [new_img_name], "cl": [cl]})
    test_mapping = pd.concat([test_mapping, new_row], ignore_index=True)
    shutil.copy(img, f"data/test_all/{new_img_name}.JPEG")
#     print(img, cl)

In [7]:
test_mapping["img"] = test_mapping.img + ".JPEG"

In [8]:
test_mapping.to_csv("data/test_mapping",index=False)

# Final evaluation

In [9]:
from torchvision.datasets import ImageFolder
import torch
import torchvision
import torchvision.transforms as transforms
import pandas as pd
import numpy as np

In [10]:
test_mapping = pd.read_csv("data/test_mapping")

In [11]:
pd.DataFrame({"a":test_mapping.img, "b":np.random.randint(0,50,len(test_mapping))}).to_csv("all_pia/results/random.csv", index=False, header=None)

OSError: Cannot save file into a non-existent directory: 'all_pia\results'

In [ ]:
import os
directory = "results/poniedzialek/all_pon/"
files = os.listdir(directory)

In [ ]:
results = pd.DataFrame()
for file in files:
    try:
        if file[-4:]==".csv":
            team_name = file[:-4]
        else:
            team_name = file
        predictions = pd.read_csv(directory+file, header=None)
        if len(predictions.columns)!=2:
            print(f"Warning: more columns for team {team_name}")
            pred_scores = predictions.merge(test_mapping,left_on=1,right_on="img")
            scores = pred_scores[2]
        else:
            pred_scores = predictions.merge(test_mapping,left_on=0,right_on="img")
            scores = pred_scores[1]

        accuracy = (scores == pred_scores.cl).sum()/len(test_mapping)
        print(len(scores), len(pred_scores))
        results = results.append({"team":team_name,"accuracy":accuracy},ignore_index=True)
    except:
        print(f"Error for {file}")
        continue
    print(team_name,accuracy)

In [ ]:
results.sort_values("accuracy",ascending=False)